In [1]:
!pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp38-cp38-linux_x86_64.whl size=5997345 sha256=fb9fb7ce87c65cf3a7f1945c36469f79468f3aa277e7455a570d05db0eb5310b
  Stored in directory: /root/.cache/pip/wheels/f3/35/48/0b9a7076995eea5ea64a7e4bc3f0f342f453080795276264e7
Successfully built mpi4py


In [4]:
!unzip ./archive.zip

In [2]:
!mkdir weights
!touch cw_mpi.py

### Ссылка на используемый датасет


https://www.kaggle.com/datasets/muniryadi/cat-vs-rabbit

In [6]:
!mpirun -np 4 --allow-run-as-root --use-hwthread-cpus --oversubscribe python cw_mpi.py

@@@@@@@@@@@@@@@@@@@@@@@@ -- TRAIN -- @@@@@@@@@@@@@@@@@@@@@@@@
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
  0%|          | 0.00/9.83M [00:00<?, ?B/s]/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter '

In [7]:
!cat cw_mpi.py

import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets, transforms, models
from torchvision.datasets.vision import data
from torchvision.transforms import ToTensor
import os
import pandas as pd
from PIL import Image
import cv2
from mpi4py import MPI
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

def train(model, criterion, optimizer, dataloader, sizes, my_rank, epochs=10):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        score = 0
        if my_rank == 0:
            iteration = 0
            for image, label in tqdm(dataloader['train']):
                comm.send(image, dest=iteration % (p - 1) + 1, tag=0)
                comm.send(label, dest=iteration % (p - 1) + 1, tag=1)
                iteration += 1
        if my_rank != 0:
            size = 0
            for i in range(len(dataloader['train'])):
                if i % (p - 1) + 1 == my_rank:
                    image 